<a href="https://colab.research.google.com/github/Yundo37/automation-portfolio-Yundo/blob/main/automation_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1) 깃허브 가져오기
!git clone https://github.com/Yundo37/automation-portfolio-Yundo.git
%cd automation-portfolio-Yundo

In [ ]:
#2) 패키지 설치
!pip install -q -r requirements.txt

In [ ]:
#3) tests/ 스캐폴딩
!mkdir -p tests reports
%%writefile tests/__init__.py
# empty

In [ ]:
#4) tests/test_e2e.py
%%writefile tests/test_e2e.py
import time
from selenium.webdriver.common.by import By
from pageObjects.HomePage import HomePage
from utilities.BaseClass import BaseClass

class TestOne(BaseClass):
    def test_e2e(self):
        log = self.getLogger()
        print("[E2E] 홈 진입 및 상점 이동")
        homePage = HomePage(self.driver)
        checkoutpage = homePage.shopItems()

        log.info("getting all the product title")
        print("[E2E] 상품 목록 수집")
        products = checkoutpage.getProductTitles()

        for product in products:
            productName = product.find_element(By.XPATH, "div/h4/a").text
            log.info(productName)
            print(f"[E2E] 발견한 상품: {productName}")
            if productName == "Blackberry":
                checkoutpage.getProductFooter(product).click()
                print("[E2E] Blackberry 담기 완료")
                break

        print("[E2E] 카트 페이지로 이동")
        self.driver.find_element(By.CSS_SELECTOR, "a[class*=btn-primary]").click()

        print("[E2E] 체크아웃 페이지 진입")
        confirmPage = checkoutpage.CheckOutItems()

        log.info("Entering country name is ind")
        print("[E2E] 국가 입력: ind")
        confirmPage.enterCountry("ind")
        self.verifyLinkPresence("India")

        print("[E2E] 국가 'India' 선택 및 약관 동의")
        confirmPage.selectCountry().click()
        confirmPage.checkTerms().click()
        confirmPage.submitOrder().click()

        successText = confirmPage.getSuccessMessage()
        log.info("Text received from application is" + successText)
        print(f"[E2E] 완료 메시지: {successText}")
        assert "Success! Thank you!" in successText
        time.sleep(1)

In [ ]:
#5) tests/test_HomePage.py (의도적 fail 1건)
%%writefile tests/test_HomePage.py
import pytest
from pageObjects.HomePage import HomePage
from testData.HomePageData import HomePageData
from utilities.BaseClass import BaseClass

class TestHomePage(BaseClass):
    def test_formSubmission(self, getData):
        log = self.getLogger()
        homepage = HomePage(self.driver)

        print(f"[FORM] 입력 시작: {getData}")
        log.info("user name is" + getData["username"])

        homepage.getName().send_keys(getData["username"])
        homepage.getEmail().send_keys(getData["email"])
        homepage.getPassword().send_keys("123456")
        homepage.selectCheckbox().click()
        homepage.selectEmploymentStatus().click()
        self.selectOptionByText(homepage.getGenderDropdown(), getData["gender"])
        self.selectOptionByIndex(homepage.getGenderDropdown(), 0)

        homepage.submitForm().click()
        message = homepage.getSuccessMessage()
        print(f"[FORM] 성공 메시지: {message}")

        if getData["username"] == "OB":           # 데모용 실패
            print("[FORM] 데모용 실패 트리거")
            assert "ThisShouldFail" in message
        else:
            assert "Success" in message

        self.driver.refresh()

    @pytest.fixture(params=HomePageData.test_Homepage_data)
    def getData(self, request):
        return request.param

In [ ]:
6) tests/conftest.py (헤드리스 + Base64 스크린샷 인라인)
%%writefile tests/conftest.py
import os, base64, pytest
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager

driver = None

def pytest_addoption(parser):
    parser.addoption("--browser_name", action="store", default="chrome")

@pytest.fixture(scope="class")
def setup(request):
    global driver
    browser_name = request.config.getoption("browser_name")
    if browser_name == "chrome":
        service_obj = ChromeService(ChromeDriverManager().install())
        options = ChromeOptions()
        options.add_argument("--headless=new")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(service=service_obj, options=options)
    else:
        raise RuntimeError("Only chrome supported in Colab")

    driver.get("https://rahulshettyacademy.com/angularpractice/")
    driver.implicitly_wait(5)
    driver.maximize_window()

    request.cls.driver = driver
    yield
    driver.close()

@pytest.mark.hookwrapper
def pytest_runtest_makereport(item):
    pytest_html = item.config.pluginmanager.getplugin('html')
    outcome = yield
    report = outcome.get_result()
    extra = getattr(report, 'extra', [])

    if report.when in ('call','setup'):
        xfail = hasattr(report, 'wasxfail')
        if (report.skipped and xfail) or (report.failed and not xfail):
            file_name = report.nodeid.replace("::", "_") + ".png"
            file_path = os.path.join("reports", file_name)
            _capture_screenshot(file_path)
            with open(file_path, "rb") as f:
                b64 = base64.b64encode(f.read()).decode()
            html = (
                f'<div><img src="data:image/png;base64,{b64}" alt="screenshot" '
                f'style="width:304px;height:228px;" '
                f'onclick="window.open(this.src)" align="right"/></div>'
            )
            extra.append(pytest_html.extras.html(html))
    report.extra = extra

def _capture_screenshot(path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    driver.get_screenshot_as_file(path)

In [ ]:
#7) 실행 및 리포트 생성 (HTML + 스크린샷)
# 1) 테스트 실행 & HTML 리포트 생성
!pytest -q --html=reports/report.html --self-contained-html

# 2) HTML 리포트 보기
from IPython.display import IFrame, Image, display
import glob

print("\n=== 📄 HTML 리포트 ===")
display(IFrame(src='reports/report.html', width='100%', height=600))

# 3) 실패 스크린샷 원본 보기 (있을 경우)
png_files = glob.glob('reports/*.png')
if png_files:
    print("\n=== 🖼 실패 스크린샷 원본 ===")
    for img_path in png_files:
        display(Image(filename=img_path))
else:
    print("\n(실패 스크린샷 없음)")